In [1]:
class User:
    def __init__(self, name, role):
        self.name = name
        self.role = role
        self.pos_system = None  # To be assigned during runtime respectively

    def display_actions(self):
        pass # override in child class

    def select_action(self):
        pass # override in child class

In [2]:
class Guest(User):
    def __init__(self, name):
        super().__init__(name, role="Guest")
        self.orders = []
        self.amount = 0

    def display_actions(self):
        print("\nAvailable actions:")
        print("1: Place takeaway order")
        print("2: View my order status")
        print("3: Log out")

    def select_action(self):
        while True: # error handling 
            self.display_actions()
            
            try:
                choice = input("\n➡️ Select action (1-3): ")

                if choice == "1":
                    self.place_takeaway_order()
                elif choice == "2":
                    self.view_order_status()
                elif choice == "3":
                    print("\nReturning to main menu...")
                    return "exit"
                else:
                    print("❌ Invalid choice. Please try again.")
            except ValueError:
                print("❌ Invalid input. Please enter a number (1, 2, or 3).")

    def place_takeaway_order(self):
        order = Order(self.pos_system.current_order_id, self.name, self.pos_system, is_takeaway = True) # define order inside the method
        self.pos_system.current_order_id += 1 # increase order id by 1 for every new orders

        while True:
            print("\n=== ☕️ Today's Menu 🍔 ===")
            self.pos_system.display_menu()
            choice = input("\n➡️ Enter item number to order or 'done' to finish order: ")

            if choice.lower() == 'done':
                break

            item = self.pos_system.get_menu_item(choice)

            if item:
                order.add_item(item) # add selected item to order
                print(f"\n✅ Added {item['name']} to your order!")

        if order.items:
            self.pos_system.orders.append(order) # add order to order dictionary
            print("\n✅ Order placed successfully!")
            order.display_order()
        else:
            print("\n❗️Order cancelled or no items selected.")

    def view_order_status(self):
        my_orders = [order for order in self.pos_system.orders if order.name == self.name]

        if not my_orders:
            print("\n❗️No orders found.")
            return

        print("\nYour Orders:")
        for order in my_orders:
            order.display_order() # method defined in order class



In [3]:
class Waiter(User):
    def __init__(self, name):
        super().__init__(name, role = "Waiter")
        self.orders = []
        self.revenue = 0 #revenue of all orders registered by waiter
        
    def display_actions(self):
        print("\nAvailable actions:")
        print("1: Place new order")
        print("2: View all my orders")
        print("3: Cancel Order")
        print("4. Log out")
        
    def select_action(self):
        while True:
            self.display_actions()
            
            try: 
                choice = input("\n➡️ Select action (1-4): ")
            
                if choice == "1":
                    self.place_new_order()
                elif choice == "2":
                    self.view_all_orders()
                elif choice == "3":
                    self.cancel_order()
                elif choice == "4":
                    print("\nReturning to main menu...")
                    return "exit"
                else:
                    print("❌ Invalid choice. Please try again.")
            except ValueError:
                print("❌ Invalid input. Please enter a number (1, 2, 3, or 4).")
        
    def place_new_order(self):
        table_number = input("➡️ Enter table # (1-10) or 100 for takeaway") 
        order = Order(self.pos_system.current_order_id, self.name, self.pos_system, table_number = table_number)
        self.pos_system.current_order_id += 1 
        
        while True: 
            if not (1 <= int(table_number) <= 10 or int(table_number) == 100):
                print("❗️ Invalid table #. Please enter a number between 1-10 or 100.")  
                return
        
            table_number = int(table_number) #enter table number
            
            print("=== ☕️ Today's Menu 🍔 ===")
            self.pos_system.display_menu()
            choice = input("\n➡️ Enter item number to order or 'done' to finish order: ")
            
            if choice.lower() == 'done':
                break

            item = self.pos_system.get_menu_item(choice) 

            if item:
                order.add_item(item)
                print(f"\n✅ Added {item['name']} to the order on table #{order.table_number}!")

        if order.items:
            self.pos_system.orders.append(order) # add order item with attributes and items to order dictionary
            print(f"\n✅ Order placed successfully on table #{order.table_number}!")
            order.display_order()
        else:
            print("\n❗️ Order cancelled or no items selected.") 
            
    
    def view_all_orders(self):
        my_orders = [order for order in self.pos_system.orders if order.name == self.name]

        if not my_orders:
            print("\n❗️ No orders found.")
            return

        print("\nYour Orders:")
        if my_orders:
            for order in my_orders:
                order.display_order()
        else: 
            print("\n❗️ You have no orders.")
            
          
    def cancel_order(self):
        if not self.pos_system.orders:
            print("\n❗️ No orders to cancel.")
            return

        print("\n=== Current Orders ===")
        for order in self.pos_system.orders:
            print(f"Order ID: {order.order_id}, Ordered by: {order.name}, Table: #{order.table_number}")
            print(f"\nItems: {[item['name'] for item in order.items]}")
            
        while True:
            try:
                choice = input("\n➡️ Enter Order ID to cancel (or 'done' to exit): ")

                if choice.lower() == 'done':
                    print("Exiting order cancellation.")
                    break
                
                # find order with order ID
                order_id = int(choice)
                order_to_cancel = next((order for order in self.pos_system.orders if order.order_id == order_id and order.name == self.name), None)

                if order_to_cancel:
                    self.pos_system.orders.remove(order_to_cancel) # removes selected order from order dictionary
                    print(f"\n✅ Order {order_id} has been canceled successfully!")
                else:
                    print(f"❌ You do not have permission to cancel Order {order_id} or it does not exist.")
            except ValueError:
                print("❌ Invalid input. Please enter a valid Order ID or 'done' to exit.")
        

In [4]:
class KitchenStaff(User):
    def __init__(self,name):
        super().__init__(name, role = "KitchenStaff")
        
    def display_actions(self):
        print("\nAvailable actions:")
        print("1: View food orders")
        print("2: Mark food order as complete")
        print("3: Log out")
        
    def select_action(self):
        while True:
            self.display_actions()
            
            try:
                choice = input("\n➡️ Select action (1-3): ")
            
                if choice == "1":
                    self.view_food_orders()
                elif choice == "2":
                    self.complete_food_item()
                elif choice == "3":
                    print("\nReturning to main menu...")
                    return "exit"
                else:
                    print("❌ Invalid choice. Please try again")
            except ValueError:
                print("❌ Invalid input. Please enter a number (1, 2, or 3).")
        
    def merge_sort(self, orders):
        if len(orders) <= 1:
            return orders

        mid = len(orders) // 2 #splits list into two
        left_half = self.merge_sort(orders[:mid])
        right_half = self.merge_sort(orders[mid:])

        return self.merge(left_half, right_half) #merges two halves again
    
    
    def merge(self, left, right):
        sorted_orders = []
        i = j = 0

        while i < len(left) and j < len(right): 
            left_order = left[i]
            right_order = right[j]
        
            # Sort logic: prioritize takeaway first, then order_id
            # If takeaway status is different, prioritize takeaway orders first
            if left_order.is_takeaway != right_order.is_takeaway:
                if left_order.is_takeaway:  # Takeaway comes before dine-in
                    sorted_orders.append(left_order)
                    i += 1
                else:
                    sorted_orders.append(right_order)
                    j += 1
            else:
                # If takeaway status is the same, sort by order_id
                if left_order.order_id <= right_order.order_id:
                    sorted_orders.append(left_order)
                    i += 1
                else:
                    sorted_orders.append(right_order)
                    j += 1

        # Add any remaining elements from the two halves
        sorted_orders.extend(left[i:])
        sorted_orders.extend(right[j:])

        return sorted_orders

        
        
    def view_food_orders(self):
        print("\n=== Food Orders ===") # view all food orders that are not complete, sort to prioritize takeaway orders
        
        pending_food_orders = [order for order in self.pos_system.orders if any(item.get('category') == 'food' for item in order.items) and order.order_status != 'Complete']
        sorted_pending_food_orders = self.merge_sort(pending_food_orders)
                        
        for order in sorted_pending_food_orders:
            food_items = [item for item in order.items if item.get('category') == 'food']   
            
            if food_items:
                print(f"\nOrder ID: {order.order_id}, Table: #{order.table_number} - Takeaway?: {order.is_takeaway}")
                for item in food_items:
                    print(f"- {item['name']} - Status: {item['status']}")
            else:
                print("\nThere are no pending food orders.")
   
    
    def complete_food_item(self):
        if not self.pos_system.orders:
            print("\nThere are no orders to complete.")
            return
        
        self.view_food_orders()
        
        try:
            order_id = int(input("➡️ Enter the order ID: "))
        except ValueError:  
            print("❌ Invalid input. Please enter a valid order ID.")
            return
        
        # find order by order ID 
        order = next((order for order in self.pos_system.orders if order.order_id == order_id), None)
        
        if order is None:
            print(f"\nOrder with ID {order_id} does not exist.")
            return
        
        if order.order_status == "Complete":
            print(f"\nOrder {order_id} has already been marked as complete.")
            return
        
        for order in self.pos_system.orders:
            if order.order_id == order_id:
                food_items = [item for item in order.items if item.get('category') == 'food']  
                if not food_items:
                    print("❗️ No food items in this order.")
                    return
        
                print("\nFood items in this order:")
                for i, item in enumerate(food_items, 1):
                    print(f"{i}. {item['name']} - Status: {item['status']}")
                    
                try:
                    item_index = int(input("➡️ Enter the item number to mark as complete: ")) - 1
                    if 0 <= item_index < len(food_items):
                        order.complete_item(item_index)
                        print(f"✅ Marked {food_items[item_index]['name']} as completed.")
                    else:
                        print("❌ Invalid item selection.")
                except ValueError:
                    print("❌ Invalid input. Please enter a valid number.")
                return
                
        print("❗️ Order not found or already completed.")
            

In [5]:
class BarStaff(User):
    def __init__(self,name):
        super().__init__(name, role = "BarStaff")
        
    def display_actions(self):
        print("\nAvailable actions:")
        print("1: View drink orders")
        print("2: Mark drink order as complete")
        print("3: Log out")
        
    def select_action(self):
        while True:
            self.display_actions()
            
            try:
                choice = input("\n➡️ Select action (1-3): ")
                
                if choice == "1":
                    self.view_drink_orders()
                elif choice == "2":
                    self.complete_drink_item()
                elif choice == "3":
                    print("\nReturning to main menu...")
                    return "exit"
                else:
                    print("❌ Invalid choice. Please try again")
            except ValueError:
                print("❌ Invalid input. Please enter a number (1, 2, or 3).")
        
    def merge_sort(self, orders):
        if len(orders) <= 1:
            return orders

        mid = len(orders) // 2 #splits list into two
        left_half = self.merge_sort(orders[:mid])
        right_half = self.merge_sort(orders[mid:])

        return self.merge(left_half, right_half) #merges two halves again
    
    
    def merge(self, left, right): 
        sorted_orders = []
        i = j = 0

        while i < len(left) and j < len(right):
            left_order = left[i]
            right_order = right[j]
        
            # Sort logic: prioritize takeaway first, then order_id
            # If takeaway status is different, prioritize takeaway orders first
            if left_order.is_takeaway != right_order.is_takeaway:
                if left_order.is_takeaway:  # Takeaway comes before dine-in
                    sorted_orders.append(left_order)
                    i += 1
                else:
                    sorted_orders.append(right_order)
                    j += 1
            else:
                # If takeaway status is the same, sort by order_id
                if left_order.order_id <= right_order.order_id:
                    sorted_orders.append(left_order)
                    i += 1
                else:
                    sorted_orders.append(right_order)
                    j += 1

        # Add any remaining elements from the two halves
        sorted_orders.extend(left[i:])
        sorted_orders.extend(right[j:])

        return sorted_orders    
        
    def view_drink_orders(self):
        print("\n=== Drink Orders ===")# view all drink orders that are not complete, sort to prioritize takeaway orders
        
        pending_drink_orders = [order for order in self.pos_system.orders if any(item.get('category') == 'drinks' for item in order.items) and order.order_status != 'Complete']
        sorted_pending_drink_orders = self.merge_sort(pending_drink_orders)
                        
        for order in sorted_pending_drink_orders:
            drink_items = [item for item in order.items if item.get('category') == 'drinks']   
            
            if drink_items:
                print(f"\nOrder ID: {order.order_id}, Table: #{order.table_number} - Takeaway?: {order.is_takeaway}")
                for item in drink_items:
                    print(f"- {item['name']} - Status: {item['status']}")
            else:
                print("\nThere are no pending drink orders.")                      
   
    
    def complete_drink_item(self):
        if not self.pos_system.orders:
            print("\nThere are no orders to complete.")
            return

        self.view_drink_orders()
        
        try:
            order_id = int(input("➡️ Enter the order ID: "))
        except ValueError:  
            print("❌ Invalid input. Please enter a valid order ID.")
            return
        
        order = next((order for order in self.pos_system.orders if order.order_id == order_id), None)
        
        if order is None:
            print(f"\nOrder with ID {order_id} does not exist.")
            return
        
        if order.order_status == "Complete":
            print(f"\nOrder {order_id} has already been marked as complete.")
            return
        
        for order in self.pos_system.orders:
            if order.order_id == order_id:
                drink_items = [item for item in order.items if item.get('category') == 'drinks']  
                if not drink_items:
                    print("❗️ No drink items in this order.")
                    return
        
                print("\nDrink items in this order:")
                for i, item in enumerate(drink_items, 1):
                    print(f"{i}. {item['name']} - Status: {item['status']}")
                    
                try:
                    item_index = int(input("➡️ Enter the item number to mark as complete: ")) - 1 # assuming item index starts at 0
                    if 0 <= item_index < len(drink_items):
                        order.complete_item(item_index)
                        print(f"✅ Marked {drink_items[item_index]['name']} as completed.")
                    else:
                        print("❌ Invalid item selection.")
                except ValueError:
                    print("❌Invalid input. Please enter a valid number.")
                return
                
        print("❗️ Order not found or already completed.")
            

In [6]:
class Manager(User):
    def __init__(self, name):
        super().__init__(name, role="Manager")

    def display_actions(self):
        print("\nAvailable actions:")
        print("1: Add menu item")
        print("2: Remove menu item")
        print("3: View sales report")
        print("4: Cancel an order")
        print("5: Log out")

    def select_action(self):
        while True:
            self.display_actions()
            
            try:
                choice = input("\n➡️ Select action (1-5): ")

                if choice == "1":
                    self.add_menu_item()
                elif choice == "2":
                    self.remove_menu_item()
                elif choice == "3":
                    self.view_sales_report()
                elif choice == "4":
                    self.cancel_order()
                elif choice == "5":
                    print("\nReturning to main menu...")
                    return "exit"
                else:
                    print("❌ Invalid choice. Please try again.")
            except ValueError:
                print("❌Invalid input. Please enter a number (1, 2, 3, 4 or 5).")

    def add_menu_item(self):
        category = input("➡️ Enter category (food/drinks): ").strip().lower()
        if category not in self.pos_system.menu_items: 
            print("❌ Invalid category. Please choose either 'food' or 'drinks'.")
            return
        
        name = input("➡️ Enter item name: ").strip()

        try:
            price = float(input("➡️ Enter item price: "))
            if price < 0: 
                print("❗️ Price cannot be negative. Please try again.")
                return
        except ValueError:
            print("❌ Invalid input. Please enter a valid price (e.g., 9.99).")
            return
            
        self.pos_system.menu_items[category].append({'name': name, 'price': price, 'category': category})
        print(f"✅ {name} added to {category} menu at {price:.2f}€.")

    def remove_menu_item(self):
        print("\n=== ☕️ Current Menu 🍔 ===")
        self.pos_system.display_menu()
        all_items = self.pos_system.menu_items['food'] + self.pos_system.menu_items['drinks']
        
        try:
            choice = int(input("\n➡️ Enter the index of the item you want to remove: "))
            
            # Check if the choice is within range
            if 1 <= choice <= len(all_items): 
                removed_item = all_items.pop(choice)
                print(f"\n✅ Removed {removed_item['name']} from the menu!")
                
                # Update the menu display after removal
                self.pos_system.menu_items['food'] = all_items[:len(self.pos_system.menu_items['food'])]
                self.pos_system.menu_items['drinks'] = all_items[len(self.pos_system.menu_items['food']):]
                
                print("\n=== ☕️ New Menu 🍔 ===")
                self.pos_system.display_menu()

            else:
                print("\n❌ Invalid selection. Please try again.")
        
        except ValueError:
            print("\n❌ Invalid input. Please enter a valid number.")

    def view_sales_report(self):
        print("\n=== 💸 Sales Report 💸 ===")
        total_sales = sum(sum(item['price'] for item in order.items) for order in self.pos_system.orders)
        print(f"Total Sales: {total_sales:.2f}€")
        
        total_food = sum(sum(item['price'] for item in order.items if item.get('category') == 'food') for order in self.pos_system.orders)
        print(f"Total Sales Food: {total_food:.2f}€")
        
        total_drinks = sum(sum(item['price'] for item in order.items if item.get('category') == 'drinks') for order in self.pos_system.orders)
        print(f"Total Sales Drinks: {total_drinks:.2f}€")
        
    def cancel_order(self):
        if not self.pos_system.orders:
            print("\n❗️ No orders to cancel.")
            return

        print("\n=== Current Orders ===")
        for order in self.pos_system.orders:
            print(f"Order ID: {order.order_id}, Ordered by: {order.name}, Table: #{order.table_number}")
            print(f"\nItems: {[item['name'] for item in order.items]}")
            
        while True:
            try:
                choice = input("\n➡️ Enter Order ID to cancel (or 'done' to exit): ")

                if choice.lower() == 'done':
                    print("Exiting order cancellation.")
                    break

                order_id = int(choice)
                order_to_cancel = next((order for order in self.pos_system.orders if order.order_id == order_id), None)

                if order_to_cancel:
                    self.pos_system.orders.remove(order_to_cancel)
                    print(f"\n✅ Order {order_id} has been canceled successfully!")
                    break
                else:
                    print(f"❗️ No order found with Order ID: {order_id}")
            except ValueError:
                print("❌ Invalid input. Please enter a valid Order ID or 'done' to exit.")


In [7]:
class Order:
    def __init__(self, order_id, ordered_by, pos_system, table_number = None, is_takeaway = False):
        self.order_id = order_id
        self.name = ordered_by
        self.table_number = table_number if table_number else (100 if is_takeaway else None)
        self.is_takeaway = is_takeaway
        self.items = [] # list of ordered items with name, price and status
        self.order_status = "accepted" # default order status
        self.pos_system = pos_system

    def add_item(self, item):
        item['status'] = "pending"
        self.items.append(item)
        
    def remove_item(self, item):
        self.items.remove(item)
    
    def display_order(self):
        print(f"\nOrder ID: {self.order_id}, Ordered by: {self.name},"
              f"\nTable: #{self.table_number}, Order Status: {self.order_status}")
        
        food_items = [item for item in self.items if item in self.pos_system.menu_items['food']]
        drink_items = [item for item in self.items if item in self.pos_system.menu_items['drinks']]
        
        # Display food items
        print("\n🍔 Food Items:")
        if food_items:
            for i, item in enumerate(food_items, 1):
                print(f"{i}. {item['name']} - {item['price']:.2f}€ - {item['status']}")
        else: 
            print("❗️ No food items in this order")

        #Display drink items
        print("\n☕️ Drink Items:")
        if drink_items:
            for i, item in enumerate(drink_items, 1):
                print(f"{i}. {item['name']} - {item['price']:.2f}€ - {item['status']}")
        else: 
            print("❗️ No drink items in this order")

        print(f"Total: {sum(item['price'] for item in self.items):.2f}€")
    
    def update_status(self, new_status):
        self.order_status = new_status
        
    def complete_item(self, item_index):
        if 0 <= item_index < len(self.items):
            self.items[item_index]['status'] = "completed"
            print(f"✅ Marked {self.items[item_index]['name']} as completed.")
        else:
            print("❌ Invalid item selection.")
        
        # Check if all items are completed to mark the whole order as completed
        if all(item['status'] == "completed" for item in self.items):
            self.order_status = "completed"
            print(f"✅ Order {self.order_id} has been marked as completed!")

In [ ]:
class POSSystem:
    def __init__(self):
        self.orders = []
        self.current_order_id = 1
        self.menu_items = {
            'food': [
                {'name': 'Burger', 'price': 12.99, 'category': 'food'},
                {'name': 'Salad', 'price': 8.99, 'category': 'food'},
                {'name': 'Sandwich', 'price': 9.99, 'category': 'food'}
            ],
            'drinks': [
                {'name': 'Coffee', 'price': 3.99, 'category': 'drinks'},
                {'name': 'Tea', 'price': 2.99, 'category': 'drinks'},
                {'name': 'Smoothie', 'price': 5.99, 'category': 'drinks'}
            ]
        }

    def display_menu(self):
        print("\n🍔 Food Items:")
        for i, item in enumerate(self.menu_items['food'], 1):
            print(f"{i}. {item['name']} - {item['price']:.2f}€")

        print("\n☕️ Drink Items:")
        for i, item in enumerate(self.menu_items['drinks'], len(self.menu_items['food']) + 1):
            print(f"{i}. {item['name']} - {item['price']:.2f}€")

    def get_menu_item(self, item_number):
        try:
            item_number = int(item_number)
            total_items = len(self.menu_items['food']) + len(self.menu_items['drinks'])
            
            if 1 <= item_number <= len(self.menu_items['food']):
                return self.menu_items['food'][item_number - 1]
            elif len(self.menu_items['food']) < item_number <= (len(self.menu_items['food']) + 
                                                                len(self.menu_items['drinks'])):
                return self.menu_items['drinks'][item_number - len(self.menu_items['food']) - 1]
            else:
                raise ValueError(f"❌ Invalid menu item number. Please select a number from 1 to {total_items}.")
        except ValueError:
            print("❌ Invalid input. Please enter a number corresponding to a menu item.")
            return None

    def run(self):
        try:
            while True:
                print("\n=== Cafe POS System ===")
                print("1: Guest")
                print("2: Waiter")
                print("3: Kitchen Staff")
                print("4: Bar Staff")
                print("5: Manager")
                print("6: Exit")

                role = input("\n➡️ Enter role number (1-6): ")

                if role == "6":
                    print("Thank you for using the Cafe POS System!")
                    break

                name = input("➡️ Enter your name: ")

                user = None
                if role == "1":
                    user = Guest(name)
                elif role == "2":
                    user = Waiter(name)
                elif role == "3":
                    user = KitchenStaff(name)
                elif role == "4":
                    user = BarStaff(name)
                elif role == "5":
                    user = Manager(name)
                else:
                    print("This role is not implemented yet. Try again.")
                    continue

                if user:
                    user.pos_system = self  # Assign the POSSystem instance to the user
                    while True:
                        action_result = user.select_action()
                        if action_result == "exit":
                            break
        except Exception as e:
            print(f"❗️ An unexpected error occured: {e}")


if __name__ == "__main__":
    pos = POSSystem()
    pos.run()



=== Cafe POS System ===
1: Guest
2: Waiter
3: Kitchen Staff
4: Bar Staff
5: Manager
6: Exit

➡️ Enter role number (1-6): 1
➡️ Enter your name: Sarah

Available actions:
1: Place takeaway order
2: View my order status
3: Log out

➡️ Select action (1-3): 1

=== ☕️ Today's Menu 🍔 ===

🍔 Food Items:
1. Burger - 12.99€
2. Salad - 8.99€
3. Sandwich - 9.99€

☕️ Drink Items:
4. Coffee - 3.99€
5. Tea - 2.99€
6. Smoothie - 5.99€

➡️ Enter item number to order or 'done' to finish order: 1

✅ Added Burger to your order!

=== ☕️ Today's Menu 🍔 ===

🍔 Food Items:
1. Burger - 12.99€
2. Salad - 8.99€
3. Sandwich - 9.99€

☕️ Drink Items:
4. Coffee - 3.99€
5. Tea - 2.99€
6. Smoothie - 5.99€

➡️ Enter item number to order or 'done' to finish order: 4

✅ Added Coffee to your order!

=== ☕️ Today's Menu 🍔 ===

🍔 Food Items:
1. Burger - 12.99€
2. Salad - 8.99€
3. Sandwich - 9.99€

☕️ Drink Items:
4. Coffee - 3.99€
5. Tea - 2.99€
6. Smoothie - 5.99€

➡️ Enter item number to order or 'done' to finish order: 


➡️ Select action (1-3): 3

Returning to main menu...

=== Cafe POS System ===
1: Guest
2: Waiter
3: Kitchen Staff
4: Bar Staff
5: Manager
6: Exit

➡️ Enter role number (1-6): 5
➡️ Enter your name: Helena

Available actions:
1: Add menu item
2: Remove menu item
3: View sales report
4: Cancel an order
5: Log out

➡️ Select action (1-5): 1
➡️ Enter category (food/drinks): food
➡️ Enter item name: Soup
➡️ Enter item price: 6.99
✅ Soup added to food menu at 6.99€.

Available actions:
1: Add menu item
2: Remove menu item
3: View sales report
4: Cancel an order
5: Log out

➡️ Select action (1-5): 2

=== ☕️ Current Menu 🍔 ===

🍔 Food Items:
1. Burger - 12.99€
2. Salad - 8.99€
3. Sandwich - 9.99€
4. Soup - 6.99€

☕️ Drink Items:
5. Coffee - 3.99€
6. Tea - 2.99€
7. Smoothie - 5.99€

➡️ Enter the index of the item you want to remove: 6

✅ Removed Smoothie from the menu!

=== ☕️ New Menu 🍔 ===

🍔 Food Items:
1. Burger - 12.99€
2. Salad - 8.99€
3. Sandwich - 9.99€
4. Soup - 6.99€

☕️ Drink Items:
